In [1]:
import os
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', 100)

In [2]:
os.chdir('..')

In [3]:
!pwd

/Users/A.IVA/Documents/jupyter_notebooks/coursera_and_blogs/rossmann_competition


### 1 Load data

In [4]:
data_dir = os.path.join('..', 'data', 'rossmann-store-sales', 'source')
dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')


df_store = pd.read_csv( os.path.join(data_dir, 'store.csv'))
df_train = pd.read_csv( os.path.join(data_dir, 'train.csv'), parse_dates=[2], low_memory=False)
df_test = pd.read_csv( os.path.join(data_dir, 'test.csv'), parse_dates=[3], date_parser=dateparse)

In [7]:
df_store.shape, df_train.shape, df_test.shape

((1115, 10), (1017209, 9), (41088, 8))

In [6]:
display(df_store.head(2))
display(df_train.head(2))
display(df_test.head(2))

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0


### 2 Play with data

##### 2.1 dtypes

In [9]:
display(df_store.dtypes)
display(df_train.dtypes)
display(df_test.dtypes)

Store                          int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object

Store                     int64
DayOfWeek                 int64
Date             datetime64[ns]
Sales                     int64
Customers                 int64
Open                      int64
Promo                     int64
StateHoliday             object
SchoolHoliday             int64
dtype: object

Id                        int64
Store                     int64
DayOfWeek                 int64
Date             datetime64[ns]
Open                    float64
Promo                     int64
StateHoliday             object
SchoolHoliday             int64
dtype: object

##### 2.2 nans

In [1]:
# Check columns for `nan` values
display(df_store.isnull().sum(axis = 0))
display(df_train.isnull().sum(axis = 0))
display(df_test.isnull().sum(axis = 0))

NameError: name 'df_store' is not defined

##### 2.3 unique values

In [10]:
df_test['Open'].unique()

array([ 1., nan,  0.])

In [11]:
for col in ['DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday']:
    print('{0:20s} \t {1:100s} \t {2:100s}'.format(col, str(sorted(df_train[col].unique())), str(sorted(df_test[col].unique()))))

DayOfWeek            	 [1, 2, 3, 4, 5, 6, 7]                                                                                	 [1, 2, 3, 4, 5, 6, 7]                                                                               
Open                 	 [0, 1]                                                                                               	 [1.0, nan, 0.0]                                                                                     
Promo                	 [0, 1]                                                                                               	 [0, 1]                                                                                              
StateHoliday         	 ['0', 'a', 'b', 'c']                                                                                 	 ['0', 'a']                                                                                          
SchoolHoliday        	 [0, 1]                                                               

In [12]:
for col in ['StoreType','Assortment','Promo2']:
    print('{0:20s} \t {1:s}'.format(col, str(sorted(df_store[col].unique()))))

StoreType            	 ['a', 'b', 'c', 'd']
Assortment           	 ['a', 'b', 'c']
Promo2               	 [0, 1]


##### 2.4 Unknown values of `open` column

For not all points in the test data set the store is `open`.    
For the dates when we do not know if a given store is open we see that more than `99%` of all stores are open.   
So we could assume that these stores were open, as well. 

In [13]:
tmp = df_test.copy(deep=True)
tmp['Open_1'] = df_test['Open'].apply(lambda x: x==1).astype(int)
tmp['Open_0'] = df_test['Open'].apply(lambda x: x==0).astype(int)
tmp['Open_nan'] = df_test['Open'].isnull().astype(int)

tmp.groupby(['Date'], axis=0).agg({'Open_0': sum, 'Open_1': sum,  'Open_nan': sum}).sort_values(by=['Open_nan'], ascending=[False])

,Open_0,Open_1,Open_nan
Date,,,
2015-09-17,3,852,1
2015-09-11,1,854,1
2015-09-05,0,855,1
2015-09-07,1,854,1
2015-09-08,1,854,1
2015-09-09,1,854,1
2015-09-10,0,855,1
2015-09-12,2,853,1
2015-09-14,3,852,1


In [14]:
df_test['Open'].value_counts()

1.0    35093
0.0     5984
Name: Open, dtype: int64

In [15]:
df_test[df_test['Open'].isnull()]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0


##### 2.5 `Promo2`

Check     
`Promo2`                        0    
`Promo2SinceWeek`              544   
`Promo2SinceYear`              544   

In [16]:
# both columns are simultaneously equal to null
df_store[['Promo2SinceWeek','Promo2SinceYear']][df_store['Promo2SinceWeek'].isnull() | df_store['Promo2SinceYear'].isnull()].isnull().sum(axis = 0)

Promo2SinceWeek    544
Promo2SinceYear    544
dtype: int64

In [17]:
# As expected, if Promo2=0 -> Promo2SinceYear=Promo2SinceWeek=NaN
display(df_store[df_store['Promo2SinceWeek'].isnull() | df_store['Promo2SinceYear'].isnull()]['Promo2'].unique())
display(df_store[df_store['Promo2']==0][['Promo2SinceWeek','Promo2SinceYear']].drop_duplicates())

array([0])

,Promo2SinceWeek,Promo2SinceYear
0,NaN,NaN


In [18]:
datetime.datetime(2018,1,1) + datetime.timedelta(days=20)

datetime.datetime(2018, 1, 21, 0, 0)

In [19]:
from ts_utils.preprocess import get_promo_date
# def get_promo_date(yr, week):
    
#     try:
#         date = datetime.datetime(int(yr),1,1) + datetime.timedelta(days=7*int(week))
#         date = datetime.date(date.year, date.month, 1)
#         return date
#     except (TypeError, ValueError):
#         return pd.NaT

df_store[['Promo2SinceWeek','Promo2SinceYear']].apply(lambda x: get_promo_date(x['Promo2SinceYear'], x['Promo2SinceWeek']), axis=1)[:3]

0           NaT
1    2010-04-01
2    2011-04-01
dtype: object

In [20]:
from ts_utils.preprocess import get_promo_months
# months_to_num = {
#     'Jan': 1,
#     'Feb': 2,
#     'Mar': 3,
#     'Apr': 4,
#     'May': 5,
#     'Jun': 6,
#     'Jul': 7,
#     'Aug': 8,
#     'Sept': 9,
#     'Oct': 10,
#     'Nov': 11,
#     'Dec': 12
# }

# def get_promo_months(x):
    
#     if type(x) != str:
#         return []
#     else:
#         months = x.split(',')
#         months = map(lambda x: months_to_num[x], months)
#         return list(months)

In [21]:
set(df_store[df_store['PromoInterval'].notnull()]['PromoInterval'].apply(lambda x: len(x.split(','))))

{4}

In [22]:
import itertools
import math

months = list(df_store['PromoInterval'].values)
months = filter(lambda x: type(x)==str, months)
months = map(lambda x: x.split(','), months)
months = itertools.chain(*months)
months = list(set(months))
months

['Feb',
 'May',
 'Mar',
 'Jun',
 'Jul',
 'Nov',
 'Dec',
 'Aug',
 'Sept',
 'Oct',
 'Apr',
 'Jan']

In [ ]:
df_store['PromoInterval'].apply(lambda x: get_promo_months(x))[:3]

In [24]:
from ts_utils.preprocess import dt_from_last_promotion

# def dt_from_last_promotion(curr_date, prom_months, first_prom_start):
#     """
#     `curr_date`, `first_prom_start` are dates
#     prom_months - list of integers
#     """
    
#     if prom_months and pd.notnull(first_prom_start) and pd.notnull(curr_date):

#         if curr_date.year >= first_prom_start.year: 

#             curr_year = curr_date.year
#             prev_year = curr_date.year - 1 

#             curr_year_candidates = list(map(lambda x: datetime.datetime(curr_year, x, 1), prom_months))
#             prev_year_candidate = datetime.datetime(prev_year, prom_months[-1], 1)  # look at the last promotion from the last year (if exists)

#             all_candidates = curr_year_candidates + [prev_year_candidate]

#             all_candidates = filter(lambda x: first_prom_start <= x <= curr_date, all_candidates)
#             all_candidates = list(all_candidates)

#             if all_candidates:
#                 latest_prom = max(all_candidates)
#                 return (curr_date-latest_prom).days

#     return None    

In [430]:
# curr_date = datetime.date(2018,1,1) 
# first_prom_start = datetime.date(2016,4,1) 
# prom_months = [2,4,10]
# dt_from_last_promotion(curr_date, prom_months, first_prom_start)

2017-10-01


92

In [25]:
df_store['Promo2StartFirst'] = df_store.apply(lambda x: get_promo_date(x['Promo2SinceYear'], x['Promo2SinceWeek']), axis=1)
df_store['Promo2StartMonths'] = df_store.apply(lambda x: get_promo_months(x['PromoInterval']), axis=1)
# df_store['Promo2CampDistance'] = df_store.apply(lambda x: dt_from_last_promotion(x['Date'], x['Promo2StartMonths'], x['Promo2StartFirst']), axis=1)

In [26]:
df_store.set_index('Store', inplace=True)

In [27]:
df_store.head(2)

,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Promo2StartFirst,Promo2StartMonths
Store,,,,,,,,,,,
1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,NaT,[]
2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2010-04-01,"[1, 4, 7, 10]"


In [28]:
tmp = df_train.join(df_store, on='Store')

In [29]:
tmp.head(3)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Promo2StartFirst,Promo2StartMonths
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,NaT,[]
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2010-04-01,"[1, 4, 7, 10]"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2011-04-01,"[1, 4, 7, 10]"


In [31]:
tmp['Promo2StartFirst'] = pd.to_datetime(tmp['Promo2StartFirst'])

In [32]:
tmp.dtypes

Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                  int64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth           float64
CompetitionOpenSinceYear            float64
Promo2                                int64
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
Promo2StartFirst             datetime64[ns]
Promo2StartMonths                    object
dtype: object

In [40]:
tmp['Promo2CampDistance'] = tmp.apply(lambda x: dt_from_last_promotion(x['Date'], x['Promo2StartMonths'], x['Promo2StartFirst']), axis=1)

In [41]:
tmp.shape

(1017209, 21)

In [ ]:
tmp.head(2)

##### 2.6 `CompetitionOpenSinceMonth`

Check    
`CompetitionOpenSinceMonth`    354    
`CompetitionOpenSinceYear`     354    

In [103]:
# both columns are simultaneously equal to null
df_store[['CompetitionOpenSinceMonth','CompetitionOpenSinceYear']][df_store['CompetitionOpenSinceMonth'].isnull() | df_store['CompetitionOpenSinceYear'].isnull()].isnull().sum(axis = 0)

CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
dtype: int64

In [115]:
# We could use the pyMC3 idea that I have in my mind....
df_store[df_store['CompetitionOpenSinceMonth'].isnull() | df_store['CompetitionOpenSinceYear'].isnull()].head(4)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
11,12,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
12,13,d,a,310.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"
15,16,a,c,3270.0,NaN,NaN,0,NaN,NaN,NaN
18,19,a,c,3240.0,NaN,NaN,1,22.0,2011.0,"Mar,Jun,Sept,Dec"


In [42]:
df_store['CompetitionOpenSinceMonth'].unique()

array([ 9., 11., 12.,  4., 10.,  8., nan,  3.,  6.,  5.,  1.,  2.,  7.])

In [43]:
df_store.dtypes

StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
Promo2StartFirst              object
Promo2StartMonths             object
dtype: object

In [ ]:
df_store[df_store['CompetitionOpenSinceMonth'].isnull() | df_store['CompetitionOpenSinceYear'].isnull()]

##### 2.7 `CompetitionDistance`

Check    
`CompetitionDistance`            3

In [107]:
# We could make a map between number of customers and a competition distance
df_store[df_store['CompetitionDistance'].isnull() ]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


##### 2.7 Data type conversion `DayOfWeek`

In [ ]:
df_store

In [ ]:
from sklearn import base
from collections import Counter
import numpy as np

In [116]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [110]:
weekday_encoder = LabelEncoder()
weekday_encoder.fit(df_train['DayOfWeek'])

# weekday_encoder.classes_
# weekday_encoder.transform([1,1,2,3,4,5,6,7])

LabelEncoder()

In [134]:
weekday_encoder_2 = OneHotEncoder(handle_unknown='ignore', sparse=False)
weekday_encoder_2.fit(df_train[['DayOfWeek']])

weekday_encoder_2.categories_
display(df_train[['DayOfWeek']][10000:10003])
weekday_encoder_2.transform(df_train[['DayOfWeek']][10000:10003])

,DayOfWeek
10000,4
10001,4
10002,4


array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.]])

##### 2.8 look at the data for different stores

In [137]:
len(df_train['Store'].unique())

1115

In [146]:
import matplotlib.pyplot as plt

In [155]:
# number of records per store
df_train[['Store','Date']].groupby(by=['Store']).count().rename(columns={"Date": "tot_count"}, inplace=False)['tot_count'].value_counts()

942    934
758    180
941      1
Name: tot_count, dtype: int64

In [163]:
tmp = df_train[['Store','Date']].groupby(by=['Store']).count().rename(columns={"Date": "tot_count"}, inplace=False)
type_A_stores = list(tmp[tmp['tot_count'].isin([941,942])].index)
type_B_stores = list(tmp[tmp['tot_count'] == 758].index)

In [179]:
tmp = df_train[df_train['Store'].isin(type_A_stores)].groupby(by=['Store']).agg({'Date': [min,max]})
display(tmp['Date']['min'].value_counts())
display(tmp['Date']['max'].value_counts())

tmp = df_train[df_train['Store'].isin(type_B_stores)].groupby(by=['Store']).agg({'Date': [min,max]})
display(tmp['Date']['min'].value_counts())
display(tmp['Date']['max'].value_counts())

2013-01-01    934
2013-01-02      1
Name: min, dtype: int64

2015-07-31    935
Name: max, dtype: int64

2013-01-01    180
Name: min, dtype: int64

2015-07-31    180
Name: max, dtype: int64

All stores have recorded events in the same time range but the total number of events is still different.    

In [188]:
# look only at the events/records when the store is opened.
# df_train[df_train['Open']==1].groupby(by=['Store']).agg({'Date': 'count'})['Date'].value_counts()

Look at the case when `open=1` but the number of customers is `0`.    
(The other case where `open=0` but the number of customers is `>0` is not present.)    
    
Maybe these events are related to the fact that one of the systems that is supposed to give info about this type of events was fucked up 
or there was an error in the `Open` column
From the three examples at which I looked the starnge combinations appear in the beginning/end of a long period of time of store inactivity.
I would suggest to set `open=0` in all these cases

In [195]:
print(df_train[(df_train['Open']==1) & (df_train['Customers']==0)].shape)
df_train[(df_train['Open']==1) & (df_train['Customers']==0)].sort_values(by=['Store', 'Date'])

(52, 9)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
562314,25,3,2014-02-12,0,0,1,0,0,0
561199,25,4,2014-02-13,0,0,1,0,0,0
348604,28,2,2014-09-02,0,0,1,1,0,1
347669,28,3,2014-09-03,0,0,1,1,0,1
346734,28,4,2014-09-04,0,0,1,1,0,0
407532,57,2,2014-07-01,0,0,1,1,0,0
397285,102,6,2014-07-12,0,0,1,0,0,0
386065,102,4,2014-07-24,0,0,1,0,0,1
340348,227,4,2014-09-11,0,0,1,0,0,0
990681,232,4,2013-01-24,0,0,1,1,0,0


In [44]:
df_train[(df_train['Open']==0) & (df_train['Customers']>0)].sort_values(by=['Store', 'Date']).shape

(0, 9)

In [205]:
# df_train[(df_train['Store']==25) & (df_train['Date']<'2014-02-20') & (df_train['Date']>'2014-02-01')]
# df_train[(df_train['Store']==28) & (df_train['Date']<'2014-09-15') & (df_train['Date']>'2014-08-25')]
# df_train[(df_train['Store']==102) & (df_train['Date']<'2014-07-31') & (df_train['Date']>'2014-07-01')]

Look at the timediff between records for every `Store`

In [206]:
groups = df_train.groupby(['Store'])

# .sort_values(['created_at'], ascending=[True])\

In [246]:
df_train['Date_1'] = df_train.sort_values(['Date'], ascending=[True]).groupby(['Store'])['Date'].shift(1)

In [257]:
df_train['Date_dt'] = (df_train['Date'] - df_train['Date_1']).apply(lambda x: x.days)
# df_train.sort_values(['Store','Date'])

In [258]:
df_train.head(3)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Date_1,Date_dt
0,1,5,2015-07-31,5263,555,1,1,0,1,2015-07-30,1.0
1,2,5,2015-07-31,6064,625,1,1,0,1,2015-07-30,1.0
2,3,5,2015-07-31,8314,821,1,1,0,1,2015-07-30,1.0


In [260]:
df_train['Date_dt'].value_counts()

1.0      1015914
185.0        180
Name: Date_dt, dtype: int64

We have 180 cases where we have a very big jump in time.    
Below we show that these cases belong to the `typeB` stores.    
This jump occures in the time window from `2014-06-30` to `2015-01-01`.

In [262]:
display(df_train[df_train['Store'].isin(type_A_stores)]['Date_dt'].value_counts())
display(df_train[df_train['Store'].isin(type_B_stores)]['Date_dt'].value_counts())

1.0    879834
Name: Date_dt, dtype: int64

1.0      136080
185.0       180
Name: Date_dt, dtype: int64

In [268]:
display(df_train[df_train['Date_dt']>1]['Date'].value_counts())
display(df_train[df_train['Date_dt']>1]['Date_1'].value_counts())

2015-01-01    180
Name: Date, dtype: int64

2014-06-30    180
Name: Date_1, dtype: int64

##### 2.9 Summary of all transformations